In [5]:
import os
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


In [6]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments

In [7]:
model_id = 'aeolian83/llama_ko_sft_kullm_01_experi_01'
device_map = {"": 0}

In [9]:
# 4bit QLoRA 학습을 위한 설정
bnb_4bit_compute_dtype = "bfloat16" # 코랩 무료버전에서 실행 시 "float16"를 사용하세요
bnb_4bit_quant_type = "nf4"
use_4bit = True
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [12]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map=device_map, cache_dir="/mnt/t7/.cache/huggingface/models")
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="/mnt/t7/.cache/huggingface/models")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
system_prompt = "당신은 사람들이 정보를 찾는 것을 도와주는 인공지능 비서입니다. 사용자가 질문을 하면 답변해 주세요."
instruct_01 = "1+2의 값을 알려주세요"

In [23]:
prompt = f"### instruction: {instruct_01}\n\n### output:"

In [24]:
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=True).input_ids.to("cuda")
inputs

tensor([[    1,   835, 15278, 29901, 29871, 29896, 29974, 29906, 30708, 36663,
         31286, 39227,    13,    13,  2277, 29937,  1962, 29901]],
       device='cuda:0')

In [25]:
outputs = model.generate(inputs, num_beams = 5, early_stopping=True, max_new_tokens=1024, do_sample=True, top_k=10, top_p=0.95, eos_token_id=tokenizer.eos_token_id, temperature = 0.1)

In [26]:
tokenizer.batch_decode(outputs, skip_special_tokens=False)

['<s> ### instruction: 1+2의 값을 알려주세요\n\n### output: 1+2의 값은 3입니다.\n\n이것은 간단한 산술 연산으로 구할 수 있습니다.\n\n먼저 1과 2를 더합니다:\n\n1 + 2 = 3\n\n다음으로 결과를 더 큰 숫자로 변환합니다:\n\n3 = 1 + 2\n\n마지막으로 결과를 더 큰 숫자로 변환합니다:\n\n3 = 3\n\n따라서 1+2의 값은 3입니다.\n\n이것은 1과 2를 더하는 간단한 산술 연산으로 구할 수 있는 간단한 숫자입니다.\n\n이 답변이 도움이 되었기를 바랍니다! 다른 질문이 있으시면 언제든지 문의해 주세요. 제가 도와드리겠습니다.\n\n감사합니다!\n\n[참고 자료]\n\n1. "1+2의 값을 구하는 방법". MathWorks. 2023. 3. 23.\n\n2. "1+2의 값을 구하는 방법". MathWorks. 2023. 3. 23.\n\n3. "1+2의 값을 구하는 방법". MathWorks. 2023. 3. 23.\n\n4. "1+2의 값을 구하는 방법". MathWorks. 2023. 3. 23.\n\n5. "1+2의 값을 구하는 방법". MathWorks. 2023. 3. 23.\n\n6. "1+2의 값을 구하는 방법". MathWorks. 2023. 3. 23.\n\n7. "1+2의 값을 구하는 방법". MathWorks. 2023. 3. 23.\n\n8. "1+2의 값을 구하는 방법". MathWorks. 2023. 3. 23.\n\n9. "1+2의 값을 구하는 방법". MathWorks. 2023. 3. 23.\n\n10. "1+2의 값을 구하는 방법". MathWorks. 2023. 3. 23.\n\n11. "1+2의 값을 구하는 방법". MathWorks. 2023. 3. 23.\n\n12. "1+2의 값을 구하는 방법". MathWorks. 2023. 3. 23.\n\n13. "1+2의 값을 구하는 방법". MathWorks. 2023. 3. 23.\n\n14. "1+2의 값을

In [27]:
print("토큰 개수는", len(outputs[0]))

토큰 개수는 1042


In [58]:
def format_instruction(system_prompt, input):
    prompt = f"""<s>### System: {system_prompt}\n\n### Human: {input}\n\n"""
    return prompt

In [28]:
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to("cuda")
inputs

{'input_ids': tensor([[  835, 15278, 29901, 29871, 29896, 29974, 29906, 30708, 36663, 31286,
         39227,    13,    13,  2277, 29937,  1962, 29901]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [29]:
outputs = model.generate(**inputs, max_new_tokens=1000, do_sample=True, top_k=50, top_p=0.95)

In [30]:
tokenizer.batch_decode(outputs, skip_special_tokens=False)

['### instruction: 1+2의 값을 알려주세요\n\n### output: 1 + 2의 값은 3입니다.\n\n이 대답이 도움이 되었기를 바랍니다! 다른 질문이 있으시면 언제든지 문의해 주세요. 제가 도와드리겠습니다. 제가 도와드릴 수 있도록 도와드리겠습니다. 좋은 하루 되세요!\n\n언제든지 도와드리겠습니다. 궁금한 점이 있으시면 언제든지 문의해 주세요. 도움을 드리기 위해 여기 있어요. 제가 도와드릴 수 있도록 도와드리겠습니다! 저를 찾아주셔서 감사합니다. 어떤 도움이 필요하신가요? 제가 도와드리겠습니다.\n\n도움이 필요하시면 언제든지 문의해 주세요! 제가 도와드릴 수 있도록 도와드리겠습니다. 여러분의 질문에 답해 드리겠습니다. 제가 도와드릴 수 있도록 도와드리겠습니다. 답변해 주셔서 감사합니다. 여러분에게 도움을 드리기 위해 여기 있습니다.\n\n언제든지 도와드리겠습니다. 도움이 필요하시면 언제든지 문의해 주세요! 제가 도와드릴 수 있도록 도와드리겠습니다. 제가 도와드릴 수 있도록 도와드리겠습니다! 제가 도와드릴 수 있도록 도와드리겠습니다. 어떤 질문이든 기꺼이 답변해 드릴게요! 제가 도와드릴 수 있도록 도와드리겠습니다.\n\n언제든지 도와드리겠습니다. 도움이 필요하시면 언제든지 문의해 주세요! 제가 도와드릴 수 있도록 도와드리겠습니다. 제가 도와드릴 수 있도록 도와드리겠습니다! 어떤 질문이든 기꺼이 답변해 드리겠습니다. 도움을 드릴 수 있도록 도와드리겠습니다!\n\n어떤 질문이든 기꺼이 답변해 드리겠습니다! 도움을 드릴 수 있도록 도와드리겠습니다. 여러분의 질문에 답해 드리겠습니다. 제가 도와드릴 수 있도록 도와드리겠습니다.\n\n제가 도와드릴 수 있도록 도와드리겠습니다! 도움을 드리기 위해 여기 있습니다. 제가 도와드릴 수 있도록 도와드리겠습니다! 여러분의 질문에 답해 드리겠습니다. 제가 도와드릴 수 있도록 도와드리겠습니다!\n\n언제든지 도와드리겠습니다! 도움이 필요하시면 언제든지 문의해 주세요! 제가 도와드릴 수 있도록 도와드리겠습니다.

In [31]:
print("토큰 개수는", len(outputs[0]))

토큰 개수는 1017
